In [1]:
!gdown 1BVjrYwkQ69SPXhABMpaWCwVie2OPlXL6
!unzip -q data_word -d data
# https://drive.google.com/file/d/1L1J4uFW4fIN5maq_vhR-_vTgC1GSHYGI/view?usp=drive_link


In [2]:
!gdown 1L1J4uFW4fIN5maq_vhR-_vTgC1GSHYGI
!unzip -q outputs1 -d outputs


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# OUTPUT_DIR = "/content/drive/MyDrive/outputs.zip"
#os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
# !unzip -q "/content/drive/MyDrive/outputs" -d outputs

In [3]:
!pip -q install -U pip
!pip -q install flask flask-cors pyngrok pillow

!pip -q install --no-deps bitsandbytes accelerate peft triton cut_cross_entropy unsloth_zoo
!pip -q install msgspec tyro "trl>=0.18.2,<=0.24.0,!=0.19.0" "torchao>=0.13.0"
!pip -q install "datasets>=4.2.0,<4.4.0"
!pip -q install sentencepiece protobuf "huggingface_hub>=0.34.0" hf_transfer
!pip -q install "transformers==4.56.2"
!pip -q install --no-deps unsloth
!pip -q install jiwer einops addict easydict

In [4]:
#!pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo transformers timm

from huggingface_hub import snapshot_download
snapshot_download("unsloth/DeepSeek-OCR", local_dir = "deepseek_ocr")

In [5]:
import os, logging
os.environ["UNSLOTH_WARN_UNINITIALIZED"] = "0"

# Gỡ các handler dạng Unsloth khỏi logger transformers.modeling_utils
logger = logging.getLogger("transformers.modeling_utils")
new_handlers = []
for h in logger.handlers:
    # Unsloth thường đặt handler có class name đặc trưng, ta lọc theo tên
    if "Unsloth" in h.__class__.__name__:
        continue
    new_handlers.append(h)
logger.handlers = new_handlers


In [6]:
import os, json, random, argparse, unicodedata
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple

import torch
from jiwer import wer, cer
from huggingface_hub import snapshot_download

from unsloth import FastVisionModel, is_bf16_supported
from transformers import AutoModel, Trainer, TrainingArguments

In [7]:
# ========= (1) Text utils =========
def normalize_text(s: str) -> str:
    # Giữ dấu tiếng Việt ổn định (NFC), normalize whitespace nhẹ
    s = unicodedata.normalize("NFC", s or "")
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    s = " ".join(s.split())
    return s.strip()

def set_seed(seed: int):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [8]:
LORA_DIR = "/content/outputs/outputs/handwriting_lora/lora_model"
NGROK_AUTH_TOKEN = "36pKv7Y0OCJ8uQs8aliqPO634g8_2cRFrLkFQTDq8j1jkk5Uq"  # Lấy từ ngrok.com

In [10]:
"""
Flask Server cho Google Colab - DeepSeek-OCR với LoRA adapter
Chạy trên Colab và expose qua ngrok

HƯỚNG DẪN SỬ DỤNG TRÊN COLAB:
==============================
1. Chạy cell cài đặt:
   !pip install flask flask-cors pyngrok unsloth transformers peft

2. Mount Drive (nếu model ở Drive):
   from google.colab import drive
   drive.mount('/content/drive')

3. Copy file này lên Colab và chạy

4. Copy ngrok URL và paste vào FLASK_SERVER_URL trong pic.controller.js
"""

import os
import io
import json
import unicodedata
from PIL import Image
import tempfile

import torch
from flask import Flask, request, jsonify
from flask_cors import CORS

# =============================
#  Config - THAY ĐỔI THEO MÁY BẠN
# =============================
LORA_DIR = "/content/outputs/outputs/handwriting_lora/lora_model"  # Đường dẫn tới LoRA adapter
BASE_SIZE = 1024  # Giữ nguyên 1024 để detect tốt hơn
IMAGE_SIZE = 640  # Giữ nguyên 640
CROP_MODE = True
NGROK_AUTH_TOKEN = "36pKv7Y0OCJ8uQs8aliqPO634g8_2cRFrLkFQTDq8j1jkk5Uq"  # Lấy từ https://dashboard.ngrok.com/get-started/your-authtoken
LOAD_IN_4BIT = True   # Bật 4-bit quantization để tiết kiệm VRAM

# =============================
#  Helpers
# =============================
def normalize_text(s: str) -> str:
    """Chuẩn hóa text Unicode tiếng Việt"""
    s = unicodedata.normalize("NFC", s or "")
    s = " ".join(s.split())
    return s.strip()


def ensure_white_bg(pil_img: Image.Image) -> Image.Image:
    """Ép nền trắng cho ảnh RGBA/LA"""
    if pil_img.mode in ("RGBA", "LA"):
        bg = Image.new("RGBA", pil_img.size, (255, 255, 255, 255))
        pil_img = Image.alpha_composite(bg, pil_img.convert("RGBA")).convert("RGB")
    else:
        pil_img = pil_img.convert("RGB")
    return pil_img


# =============================
#  Load Model (1 lần khi start)
# =============================
print("🔄 Loading DeepSeek-OCR model...")

# Clear CUDA cache trước khi load
torch.cuda.empty_cache()
import gc
gc.collect()

from huggingface_hub import snapshot_download
from unsloth import FastVisionModel
from transformers import AutoModel
from peft import PeftModel

# Download base model (luôn cần)
base_local = "deepseek_ocr_base"
if not os.path.exists(base_local):
    snapshot_download("unsloth/DeepSeek-OCR", local_dir=base_local)

# Kiểm tra xem có LoRA adapter không
if os.path.exists(LORA_DIR):
    # Load trực tiếp từ LoRA dir (như trong notebook inference)
    print(f"🔄 Loading model with LoRA from: {LORA_DIR}")
    model, tokenizer = FastVisionModel.from_pretrained(
        LORA_DIR,  # Load từ LoRA dir
        load_in_4bit=LOAD_IN_4BIT,
        auto_model=AutoModel,
        trust_remote_code=True,
        unsloth_force_compile=True,
        use_gradient_checkpointing="unsloth",
    )
    print("✅ Model with LoRA loaded!")
else:
    # Fallback: Load base model
    print(f"⚠️ LoRA dir not found: {LORA_DIR}")
    print("🔄 Loading base model only...")
    model, tokenizer = FastVisionModel.from_pretrained(
        base_local,
        load_in_4bit=LOAD_IN_4BIT,
        auto_model=AutoModel,
        trust_remote_code=True,
        unsloth_force_compile=True,
        use_gradient_checkpointing="unsloth",
    )
    print("✅ Base model loaded (no LoRA)!")

# Switch to inference mode
FastVisionModel.for_inference(model)
print("✅ Model loaded successfully!")

# Prompt cho OCR
PROMPT = "<image>\nHãy chép lại nguyên văn chữ viết tay trong ảnh. Giữ đúng dấu tiếng Việt. "


def run_inference(image_path: str) -> str:
    """
    Chạy inference với fallback methods
    """
    # Method 1: Thử model.infer() với save_results=True và đọc từ file
    if hasattr(model, 'infer'):
        try:
            print(f"🔍 Trying model.infer() with image: {image_path}")
            print(f"🔍 Image exists: {os.path.exists(image_path)}")

            # Kiểm tra ảnh
            from PIL import Image as PILImage
            test_img = PILImage.open(image_path)
            print(f"🔍 Image size: {test_img.size}, mode: {test_img.mode}")

            # Tạo output folder
            output_dir = "outputs_temp"
            os.makedirs(output_dir, exist_ok=True)

            # Thử với save_results=True
            output = model.infer(
                tokenizer,
                prompt=PROMPT,
                image_file=image_path,
                output_path=output_dir,
                base_size=BASE_SIZE,
                image_size=IMAGE_SIZE,
                crop_mode=CROP_MODE,
                save_results=True,  # Thử True
                test_compress=False,
            )

            print(f"🔍 model.infer() returned: type={type(output)}, value={repr(output)}")

            # Nếu output là None, thử đọc từ file
            if output is None or output == "":
                # Tìm file json trong output folder
                json_files = [f for f in os.listdir(output_dir) if f.endswith('.json')]
                if json_files:
                    with open(os.path.join(output_dir, json_files[-1]), 'r', encoding='utf-8') as f:
                        result_data = json.load(f)
                        output = result_data.get('text', result_data.get('result', str(result_data)))
                        print(f"📄 Read from JSON: {output}")

                # Tìm file txt
                txt_files = [f for f in os.listdir(output_dir) if f.endswith('.txt')]
                if txt_files and not output:
                    with open(os.path.join(output_dir, txt_files[-1]), 'r', encoding='utf-8') as f:
                        output = f.read().strip()
                        print(f"📄 Read from TXT: {output}")

            if output is not None and output != "":
                print(f"✅ Method 1 (model.infer) worked!")
                return output
            else:
                print(f"⚠️ Method 1 returned empty/None, trying alternatives...")
        except Exception as e:
            print(f"⚠️ Method 1 failed: {e}")
            import traceback
            traceback.print_exc()

    # Method 2: Thử gọi model.generate() trực tiếp theo cách DeepSeek-OCR
    try:
        print("🔍 Trying Method 2: Direct generate with image preprocessing")
        from PIL import Image as PILImage

        image = PILImage.open(image_path).convert("RGB")

        # Kiểm tra xem model có method nào khác không
        print(f"🔍 Model methods: {[m for m in dir(model) if not m.startswith('_') and callable(getattr(model, m, None))][:20]}")

        # Thử dùng model.__call__ hoặc forward với image
        if hasattr(model, 'prepare_inputs_for_generation'):
            print("🔍 Model has prepare_inputs_for_generation")

        # Lấy image tensor
        import numpy as np
        img_array = np.array(image)
        print(f"🔍 Image array shape: {img_array.shape}")

    except Exception as e:
        print(f"⚠️ Method 2 failed: {e}")
        import traceback
        traceback.print_exc()

    # Method 3: Thử chat interface nếu có
    try:
        print("🔍 Trying Method 3: chat interface")
        from PIL import Image as PILImage
        image = PILImage.open(image_path).convert("RGB")

        if hasattr(model, 'chat'):
            output = model.chat(tokenizer, image, PROMPT)
            print(f"✅ Method 3 (chat) worked: {output[:100] if output else 'None'}...")
            return output or ""
    except Exception as e:
        print(f"⚠️ Method 3 failed: {e}")

    print("❌ All methods failed, returning empty string")
    return ""


# =============================
#  Flask App
# =============================
app = Flask(__name__)
CORS(app)  # Cho phép CORS từ mọi origin


@app.route("/")
def home():
    return jsonify({
        "status": "ok",
        "message": "DeepSeek-OCR Server with LoRA adapter",
        "model": "unsloth/DeepSeek-OCR",
        "lora": LORA_DIR
    })


@app.route("/predict", methods=["POST"])
def predict():
    if "image" not in request.files:
        return jsonify({"error": "No image part in the request"}), 400

    file = request.files["image"]

    try:
        # Clear CUDA cache trước mỗi request
        torch.cuda.empty_cache()
        gc.collect()

        # Đọc ảnh từ request
        img_bytes = file.read()
        pil_img = Image.open(io.BytesIO(img_bytes))
        pil_img = ensure_white_bg(pil_img)

        # QUAN TRỌNG: Resize ảnh LÊN để model detect được
        # DeepSeek-OCR cần ảnh đủ lớn để detect text (đặc biệt khi không có LoRA)
        min_size = 640  # Tăng từ 400 lên 640 để match với IMAGE_SIZE
        original_size = pil_img.size

        if pil_img.width < min_size or pil_img.height < min_size:
            # Upscale giữ tỷ lệ, đảm bảo cạnh nhỏ nhất >= min_size
            ratio = max(min_size / pil_img.width, min_size / pil_img.height)
            new_size = (int(pil_img.width * ratio), int(pil_img.height * ratio))
            pil_img = pil_img.resize(new_size, Image.Resampling.LANCZOS)
            print(f"📐 Upscaled image: {original_size} -> {new_size}")

        # Giới hạn max size để tránh OOM
        max_size = 1024
        if pil_img.width > max_size or pil_img.height > max_size:
            ratio = min(max_size / pil_img.width, max_size / pil_img.height)
            new_size = (int(pil_img.width * ratio), int(pil_img.height * ratio))
            pil_img = pil_img.resize(new_size, Image.Resampling.LANCZOS)
            print(f"📐 Downscaled image to: {new_size}")

        # Lưu tạm để model đọc
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp:
            pil_img.save(tmp.name, format="PNG")
            tmp_path = tmp.name

        # Chạy inference với fallback methods
        output = run_inference(tmp_path)

        # Cleanup temp file
        try:
            os.unlink(tmp_path)
        except:
            pass

        # Debug: in ra output để xem format
        print(f"🔍 Raw output type: {type(output)}")
        print(f"🔍 Raw output: {output}")

        # Xử lý output theo nhiều format có thể
        ocr_text = ""
        if output is None:
            ocr_text = ""
        elif isinstance(output, str):
            ocr_text = output
        elif isinstance(output, dict):
            # Có thể là {"text": "...", ...} hoặc {"result": "..."}
            ocr_text = output.get("text") or output.get("result") or output.get("ocr_text") or str(output)
        elif isinstance(output, (list, tuple)):
            # Có thể là list of results
            if len(output) > 0:
                first = output[0]
                if isinstance(first, str):
                    ocr_text = " ".join(output)
                elif isinstance(first, dict):
                    ocr_text = first.get("text") or first.get("result") or str(first)
                else:
                    ocr_text = str(first)
        else:
            ocr_text = str(output)

        ocr_text = normalize_text(ocr_text)
        print(f"✅ Final OCR text: {ocr_text}")

        return jsonify({
            "message": "OK",
            "model_used": "deepseek_ocr_lora",
            "ocr_text": ocr_text,
            "details": []
        })

    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500


# =============================
#  Start Server with ngrok
# =============================
def run_with_ngrok():
    """Chạy Flask với ngrok tunnel"""
    from pyngrok import ngrok

    # Auth ngrok (chỉ cần 1 lần)
    if NGROK_AUTH_TOKEN:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Mở tunnel
    public_url = ngrok.connect(5000)
    print("\n" + "=" * 60)
    print("🚀 Flask Server đang chạy!")
    print(f"📡 Public URL: {public_url}")
    print("=" * 60)
    print("\n👉 Copy URL trên và paste vào FLASK_SERVER_URL trong")
    print("   pic.controller.js (thay thế http://localhost:5000)")
    print("\n")

    # Chạy Flask
    app.run(host="0.0.0.0", port=5000, debug=False)


if __name__ == "__main__":
    run_with_ngrok()
